In [7]:
search_for = 94
start_from = 0
threads = 1
thread = 0
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.0042116641998291016
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 25
all cases: 154558591
type: [1, 1, 1, 1, 94] 94
cases of this type: 78074896
10000 13.84218123705995
20000 14.850057221657423
30000 41.64108492862117
40000 70.39309954295636
50000 69.09240160722815
60000 72.74015429856325
70000 78.40870169835414
80000 74.14222342165988
90000 78.49569551505675
100000 93.45631938069975
110000 94.62742639527293
120000 101.4756477630096
130000 113.83880215897263
140000 103.88628769456527
150000 112.12938835205631
160000 114.4920606142798
170000 115.57192395168394
180000 104.73928711937239
190000 101.4196958413589
200000 112.5848247326197
210000 119.96956321220227
220000 123.76462147748431
230000 100.09399419433142
240000 111.88804936476231
250000 92.52142465307683
260000 94.09525911954051
270000 99.15127949777978
280000 90.632426683409
290000 93.83307592707956
300000 97.31805277462627
310000 91.2416063951016
3

3120000 83.33218684966069
3130000 91.35649616755974
3140000 85.73595863223382
3150000 88.15512405913694
3160000 82.58370817890585
3170000 82.62556548056953
3180000 82.71114335108257
3190000 78.97189780801382
3200000 78.37747301382288
3210000 88.54862816607755
3220000 85.85829382421171
3230000 84.86163164832347
3240000 86.32985417022921
3250000 87.85652144753115
3260000 80.22262889986274
3270000 80.35514613169292
3280000 85.61340192593671
3290000 33.66679269957806
3300000 33.47262150421605
3310000 32.855936081025845
3320000 34.641328553629656
3330000 89.77018523983942
3340000 103.13785627726402
3350000 27.211745263434295
3360000 27.956689424512685
3370000 27.812405985479575
3380000 26.8485467706244
3390000 69.79634489494212
3400000 97.5747251463837
3410000 84.80776259158601
3420000 82.62444394102207
3430000 84.72778701729418
3440000 90.71956881307582
3450000 85.51523600659313
3460000 81.87319241565677
3470000 83.73704449006924
3480000 83.50182207606693
3490000 85.29763619721813
3500000 

6280000 74.17538830674732
6290000 82.66557097343221
6300000 77.19021427450602
6310000 76.93664678467971
6320000 93.75416119285782
6330000 85.89484898628693
6340000 77.72972255698551
6350000 94.31207298973905
6360000 79.6545486610219
6370000 81.72874057065869
6380000 91.54340045349424
6390000 76.98569752928567
6400000 76.9064925456091
6410000 102.24085334432588
6420000 81.50412293342819
6430000 83.9539454495654
6440000 85.645915938773
6450000 79.86410738472836
6460000 84.16490528070258
6470000 90.8370301105195
6480000 81.54034608714795
6490000 78.17241479601213
6500000 86.41444069772432
6510000 79.9804024864752
6520000 76.95957958319539
6530000 89.18319019281313
6540000 77.4394363398706
6550000 82.27519154365963
6560000 96.13028347613627
6570000 60.42485348456879
6580000 31.91319560765991
6590000 32.13308359351411
6600000 32.03973213824193
6610000 42.13979652463907
6620000 100.07579843570838
6630000 85.32994643660219
6640000 82.7744160702511
6650000 86.68813035216748
6660000 90.21655290

9440000 75.18072092975092
9450000 71.35219586238543
9460000 70.78519790956058
9470000 76.49477982511802
9480000 74.15351316873985
9490000 73.7491511264946
9500000 69.08736929988332
9510000 71.72276184738801
9520000 73.75947653085089
9530000 78.78640043260751
9540000 46.93618069632064
9550000 13.07562046617957
9560000 13.017471200052718
9570000 13.029251239104708
9580000 25.793519918507037
9590000 98.74348924516865
9600000 75.38738700716523
9610000 65.23773669037527
9620000 73.01941922352415
9630000 74.5179428905532
9640000 70.5615326044756
9650000 72.8065724028885
9660000 73.39457358935086
9670000 70.50471733414233
9680000 72.61863241664953
9690000 75.56739475414885
9700000 72.3847734960637
9710000 77.29380197584064
9720000 85.30046346168004
9730000 81.40568977028775
9740000 82.36354571944912
9750000 85.05887836037722
9760000 82.57987009281895
9770000 83.25116934880597
9780000 84.20590612712257
9790000 81.89635144198942
9800000 85.45256714886304
9810000 85.02138515038553
9820000 89.164

12500000 78.11109035160942
12510000 77.24705151147653
12520000 81.93125450182202
12530000 78.67832647336031
12540000 74.97016955977509
12550000 84.42707344588948
12560000 81.19951337845364
12570000 86.12819262808279
12580000 51.57432552111423
12590000 25.88843794205449
12600000 26.088287912421716
12610000 26.146747441283424
12620000 41.902084925233495
12630000 101.82655897961283
12640000 77.80669453093246
12650000 75.85594570538099
12660000 81.05013987044332
12670000 78.88668229339135
12680000 81.39783891116761
12690000 79.9703213613044
12700000 73.51201157347002
12710000 76.1598176605449
12720000 75.2684593684173
12730000 77.3974805672528
12740000 82.19720201566982
12750000 83.1017275021495
12760000 73.63372027730715
12770000 70.9243951270174
12780000 72.36216053745396
12790000 66.47943638123614
12800000 66.2657766631576
12810000 73.50232445781124
12820000 70.80888937097464
12830000 73.0147647319617
12840000 68.93197263010644
12850000 75.17115775778151
12860000 58.407355989897646
1287

...

143070000 2.3167798715203802
143080000 3.3712659736713766
143090000 3.548590116764605
143100000 3.309095997492128
143110000 3.1724041125083167
143120000 3.3839483514824047
143130000 3.5803228716614575
143140000 3.745036600644171
143150000 3.105708624047862
143160000 3.173125607417259
143170000 3.255579302400291
143180000 3.2858381508949264
143190000 3.5398161409349775
143200000 2.6699151548026205
143210000 1.6223222520452407
143220000 2.3702995547818144
143230000 1.7789202375596365
143240000 0.9293652453923092
143250000 2.1173096375060876
143260000 1.9278632954440713
143270000 2.132688846233884
143280000 3.386085697812603
143290000 3.296014051224768
143300000 2.8063839657468668
143310000 3.3110481815501918
143320000 3.2266505038102067
143330000 3.5130409199433728
143340000 3.2404520015014406
143350000 3.375851818996131
143360000 3.5716687911301523
143370000 2.965915364771677
143380000 2.7722998138186736
143390000 3.0787486429015596
143400000 3.1438125216712223
143410000 3.2139566839351

145930000 1.1843750861313145
145940000 0.6543507609292202
145950000 1.891218834139804
145960000 1.4875206338465214
145970000 1.2726187472531596
145980000 1.8967303835229143
145990000 2.382018335653504
146000000 2.4116155509051214
146010000 2.1428450388410885
146020000 2.1555862637117107
146030000 2.368290427134818
146040000 2.6811982738466926
146050000 2.6861648936061857
146060000 2.0923872565768957
146070000 2.188133969706416
146080000 2.3526128114420377
146090000 2.3625632840259474
146100000 2.10400054819817
146110000 1.443032019736588
146120000 1.506018980244961
146130000 2.6781284420602454
146140000 0.749595478828907
146150000 0.5644740826601254
146160000 2.3497765660582184
146170000 2.0027273857931696
146180000 1.3701337184122668
146190000 1.6315761226988301
146200000 2.5221626329126754
146210000 2.008779003197398
146220000 2.521054404681484
146230000 1.9204808003687264
146240000 2.1339404388926426
146250000 2.3404495973552795
146260000 2.3359838573726814
146270000 2.4226757665754

148790000 1.7040676661408212
148800000 1.5570140144521991
148810000 1.6506845233449934
148820000 1.4780437061136895
148830000 1.4888773822889925
148840000 0.5596639516299367
148850000 0.5000847039189471
148860000 1.5694615704121326
148870000 1.7322881359415452
148880000 1.5675206160799464
148890000 1.6884736946297882
148900000 1.6670340436263282
148910000 1.5609286133477556
148920000 1.7840042127587992
148930000 1.0835122024683357
148940000 0.8063408725735745
148950000 0.9891966934422255
148960000 0.9406026799671053
148970000 1.8846984306381742
148980000 1.3991184938569532
148990000 1.5381787971236704
149000000 1.7087024142812093
149010000 1.8050546825322378
149020000 1.587179713676512
149030000 1.7112703370229865
149040000 1.2857044805257254
149050000 0.4774735501525601
149060000 0.5967632002008757
149070000 1.7812953440346317
149080000 1.748500975652039
149090000 1.4955782521312104
149100000 1.6439928341432344
149110000 1.7264078892974255
149120000 1.4276410222498908
149130000 1.4458

151470000 0.7633358889137506
151480000 0.7544179804663856
151490000 0.25774493290995887
151500000 0.26734633362065424
151510000 0.35493765130901334
151520000 0.5098012002461222
151530000 0.81683942636923
151540000 0.8017652751545311
151550000 0.970702007233507
151560000 0.8205927970425618
151570000 0.828651117070814
151580000 0.8491252486883734
151590000 0.49722662387413447
151600000 0.3043407552620769
151610000 0.3626154908551375
151620000 0.8365336877923343
151630000 0.8625504428293705
151640000 0.6027172245129877
151650000 0.8334484063273536
151660000 0.8349353823245765
151670000 0.7898833449467685
151680000 0.7375479810413587
151690000 0.7386987329926888
151700000 0.31421881707599425
151710000 0.332792938364466
151720000 0.676967160669744
151730000 0.7113541180550323
151740000 0.7035448040111197
151750000 0.7229892718624671
151760000 0.4224113215360046
151770000 0.17923448834731182
151780000 0.1940752545700471
151790000 0.17033261193933752
151800000 0.19453708967309527
151810000 0.

154260000 0.08655294015798304
154270000 0.08812366975351175
154280000 0.08959142647193538
154290000 0.08582741737845208
154300000 0.03083241259960334
154310000 0.06771348221427202
154320000 0.0729515540152457
154330000 0.07215630684000254
154340000 0.07186560464701387
154350000 0.032970489071289695
154360000 0.05657530555488666
154370000 0.06648995963282718
154380000 0.06380757842910952
154390000 0.05615403688536087
154400000 0.03579133787211444
154410000 0.02884519392032756
154420000 0.04743305650162696
154430000 0.04644254641554753
154440000 0.04055603610452679
154450000 0.03092559557749828
best so far: 0
type: [2, 1, 1, 47, 1] 94
cases of this type: 103823
154460000 0.009451892215702268
154470000 0.00628229203402996
154480000 0.005074070436596871
154490000 0.0049437019629147315
154500000 0.004074949587570296
154510000 0.0035593217566013335
154520000 0.0027745698458949727
154530000 0.0020557373077405823
154540000 0.0013529172950983049
154550000 0.0006080792221559418
best so far: 0
ty